<a href="https://colab.research.google.com/github/Saloni1707/Car_Racing/blob/main/car_specs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import math
import random
from dataclasses import dataclass
from typing import List,Tuple,Optional
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#basic math
@dataclass
class Vector2D:
  x:float
  y:float
  def __add__(self,other):
    return Vector2D(self.x+other.x,self.y+other.y)
  def __sub__(self,other):
    return Vector2D(self.x-other.x,self.y-other.y)
  def __mul__(self,scalar):
    return Vector2D(self.x*scalar,self.y*scalar)
  def magnitude(self):
    return math.sqrt(self.x**2+self.y**2)
  def normalize(self):
    mag = self.magnitude()
    if mag == 0:
      return Vector2D(0,0)
    return Vector2D(self.x/mag,self.y/mag)
  def rotate(self,angle_rad:float):
    cos_a=math.cos(angle_rad)
    sin_a=math.sin(angle_rad)
    return Vector2D(
        self.x*cos_a-self.y*sin_a,
        self.x*sin_a + self.y*cos_a
    )
    def to_array(self):
      return np.array([self.x,self.y])
print("Vector Math Test")
v1=Vector2D(1,0)
v2=v1.rotate(math.pi/2)
print(f"Orginal:({v1.x:.2f},{v1.y:.2f})")
print(f"Rotated:({v2.x:.2f},{v2.y:.2f})")
print(f"Magnitude: {v1.magnitude():.2f}")


Vector Math Test
Orginal:(1.00,0.00)
Rotated:(0.00,1.00)
Magnitude: 1.00


In [8]:
#car physics here
@dataclass
class car:
  def __init__(self,x:float,y:float,angle:float):
    self.position=Vector2D(x,y)
    self.angle=angle
    self.velocity=Vector2D(0,0)
    self.speed=0
    self.angular_velocity=0
    self.max_speed = 50.0 #pixels/second
    self.max_reverse_speed=-20.0
    self.acceleration=30.0
    self.deceleration=50.0
    self.turn_speed=3.0
    self.width=20
    self.length=40
    self.sensor_ranges=[]

  def update(self,dt:float,throttle:float,steering:float):
    throttle=max(-1,min(1,throttle))
    steering=max(-1,min(1,steering))
    if throttle>0: #here based on throttle inc speed
      self.speed+=self.acceleration*throttle*dt
      self.speed=min(self.speed,self.max_speed)
    elif throttle<0:
      self.speed+=self.acceleration*throttle*dt
      self.speed=max(self.speed,self.max_reverse_speed)
    else:
      if self.speed>0:
        self.speed=max(0,self.speed-self.deceleration*dt)
      else:
        self.speed=min(0,self.speed+self.deceleration*dt)

    if abs(self.speed) > 0.1:  # Only turn when moving
        self.angular_velocity = steering * self.turn_speed * (self.speed / self.max_speed)
        self.angle += self.angular_velocity * dt
        self.angle = self.angle % (2 * math.pi)  # Keep angle in [0, 2π]
    # Update position based on speed and angle
    direction = Vector2D(math.cos(self.angle), math.sin(self.angle))
    velocity = direction * self.speed
    self.position = self.position + velocity * dt
    self.velocity = velocity

  def get_state(self):
    return np.array([
        self.position.x,self.position.y,
        self.velocity.x,self.velocity.y,
        self.angle,self.angular_velocity,
        self.speed
    ])
  def reset(self,x:float,y:float,angle:float):
    self.position=Vector2D(x,y)
    self.angle=angle
    self.velocity=Vector2D(0,0)
    self.speed=0
    self.angular_velocity=0

print("Car Simulation")
my_car = car(0, 0, 0)
print(f"Initial State: {my_car.get_state()}")
for i in range(10):
    my_car.update(dt=0.1, throttle=0.5, steering=0)  # Half throttle, no steering

print(f"After 1s straight: Position({my_car.position.x:.1f}, {my_car.position.y:.1f}), Speed: {my_car.speed:.1f}")

for i in range(10):
    my_car.update(dt=0.1, throttle=0.5, steering=0.5)

print(f"After turning: Position({my_car.position.x:.1f}, {my_car.position.y:.1f}), Angle: {math.degrees(my_car.angle):.1f}°")
print()

Car Simulation
Initial State: [0 0 0 0 0 0 0]
After 1s straight: Position(8.2, 0.0), Speed: 15.0
After turning: Position(29.3, 8.5), Angle: 40.0°



In [10]:
#Our environment
@dataclass
class Environment:
  def __init__(self,width:int,height:int):
    self.width=width
    self.height=height
    self.obstacles=[]
    self.boundary=[]
    self.create_track()

  def create_track(self):
    margin=50
    self.boundary=[
        {"start":Vector2D(0,0),"end":Vector2D(self.width,0)}, #outer boundary top wall
        {"start":Vector2D(0,self.height),"end":Vector2D(self.width,self.height)}, #bottom wall
        {"start":Vector2D(0,0),"end":Vector2D(0,self.height)},#left wall
        {"start":Vector2D(self.width,0),"end":Vector2D(self.width,self.height)},#right wall

        {'start': Vector2D(margin, margin), 'end': Vector2D(self.width-margin, margin)},
        {'start': Vector2D(margin, self.height-margin), 'end': Vector2D(self.width-margin, self.height-margin)},
        {'start': Vector2D(margin, margin), 'end': Vector2D(margin, self.height-margin)},
        {'start': Vector2D(self.width-margin, margin), 'end': Vector2D(self.width-margin, self.height-margin)},

    ]


  def is_collision(self,position:Vector2D,car_width:float=20,car_length=40):
    margin=max(car_width,car_length)/2
    if(position.x<margin or position.x>self.width-margin or position.y<margin or position.y>self.height-margin):
      return True
    return False

  def get_track_distance(self,position:Vector2D):
    distances=[]
    distances.append(position.x)
    distances.append(self.width-position.x)
    distances.append(position.y)
    distances.append(self.height-position.y)
    return min(distances)
print("Test Environment")
env=Environment(width=800, height=600)
test_pos=Vector2D(100,100)
print(f"Position ({test_pos.x}, {test_pos.y}):")
print(f"Collision: {env.is_collision(test_pos)}")
print(f"Distance to boundary: {env.get_track_distance(test_pos):.1f}")
print()

Test Environment
Position (100, 100):
Collision: False
Distance to boundary: 100.0



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class steeringControl(nn.Module):
  def __init__(self,input_size:int=7,hidden_size:int=64):
    super(steeringControl,self).__init__()

    self.network=nn.Sequential(
        nn.Linear(input_size,hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size,hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size,32),
        nn.ReLU(),
        nn.Linear(32,2)
    )

  def forward(self,x):
    output=self.network(x)
    return torch.tanh(output)

print("Neural Network Test")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model=steeringControl().to(device)

test_state = torch.FloatTensor([100, 100, 10, 5, 0.5, 0.1, 15]).unsqueeze(0).to(device)
with torch.no_grad():
    output = model(test_state)
    throttle, steering = output[0].cpu().numpy()
    print(f"Input state shape: {test_state.shape}")
    print(f"Output - Throttle: {throttle:.3f}, Steering: {steering:.3f}")

Neural Network Test
cuda
Input state shape: torch.Size([1, 7])
Output - Throttle: -0.997, Steering: -0.310
